# ML - Problem Set 2: Predicting Poverty

El objetivo principal es construir un modelo predictivo de la pobreza de los hogares. Nótese que un hogar se clasifica como

\begin{equation}
    Poor = I \left( Inc < Pl \right)
\end{equation}

donde $I$ es una función indicadora que toma uno si el ingreso familiar está por debajo de una determinada línea de pobreza.

## Paso 1. Procesamiento de datos

In [1]:
# Importar librerias

import pandas as pd
from sklearn.impute import SimpleImputer

In [2]:
# Importar datos

sample = pd.read_csv('sample_submission.csv')
test_hogares = pd.read_csv('test_hogares.csv')
train_hogares = pd.read_csv('train_hogares.csv')
test_personas = pd.read_csv('test_personas.csv')
train_personas = pd.read_csv('train_personas.csv')

In [3]:
# Limpiar datos

common_columns_hogares = train_hogares.columns.intersection(test_hogares.columns) #obtener columnas comunes entre train_hogares y test_hogares
common_columns_personas = train_personas.columns.intersection(test_personas.columns) #obtener columnas comunes entre train_personas y test_personas
train_hogares = train_hogares[common_columns_hogares] #filtrar las columnas para alinear los datos
train_personas = train_personas[common_columns_personas] #filtrar las columnas para alinear los datos

columnas_deseadas = ['id', 'Orden', 'Clase', 'Dominio', 'P6020', 'P6040', 'P6050', 'P6090', 'P6100',
                     'P6210', 'P6210s1', 'P6240', 'Oficio', 'P6426', 'P6430', 'P6590', 'P6800',
                     'P6870', 'P6920', 'P7090', 'P7110', 'P7120', 'P7350', 'P7495', 'P7510s5',
                     'Pet', 'Oc', 'Des', 'Ina', 'Fex_c', 'Depto', 'Fex_dpto'] #datos mas relevantes

train_personas = train_personas[columnas_deseadas]
test_personas = test_personas[columnas_deseadas]

train_personas['P6210s1'] = train_personas['P6210s1'].where(train_personas['P6210s1'] <= 15, pd.NA) #eliminar nivel educativo 99
test_personas['P6210s1'] = test_personas['P6210s1'].where(test_personas['P6210s1'] <= 15, pd.NA) #eliminar nivel educativo 99

<ipython-input-3-4931c81a3550>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_personas['P6210s1'] = test_personas['P6210s1'].where(test_personas['P6210s1'] <= 15, pd.NA) #eliminar nivel educativo 99


In [4]:
# Imputar datos falntantes

# Crear imputadores
imputador_moda = SimpleImputer(strategy='most_frequent')  # Para variables categóricas
imputador_mediana = SimpleImputer(strategy='median')  # Para variables numéricas

# Función para imputar datos
def imputar_datos(df):
    # Separar columnas numéricas y categóricas
    columnas_numericas = df.select_dtypes(include=['number']).columns
    columnas_categoricas = df.select_dtypes(exclude=['number']).columns

    # Imputar numéricas con la mediana
    if not columnas_numericas.empty:
        df[columnas_numericas] = imputador_mediana.fit_transform(df[columnas_numericas])

    # Imputar categóricas con la moda
    if not columnas_categoricas.empty:
        df[columnas_categoricas] = imputador_moda.fit_transform(df[columnas_categoricas])

# Aplicar imputación a todos los DataFrames
imputar_datos(test_hogares)
imputar_datos(train_hogares)
imputar_datos(test_personas)
imputar_datos(train_personas)

print(train_personas.columns.tolist())
print(test_personas.columns.tolist())
print(train_hogares.columns.tolist())
print(test_hogares.columns.tolist())

['id', 'Orden', 'Clase', 'Dominio', 'P6020', 'P6040', 'P6050', 'P6090', 'P6100', 'P6210', 'P6210s1', 'P6240', 'Oficio', 'P6426', 'P6430', 'P6590', 'P6800', 'P6870', 'P6920', 'P7090', 'P7110', 'P7120', 'P7350', 'P7495', 'P7510s5', 'Pet', 'Oc', 'Des', 'Ina', 'Fex_c', 'Depto', 'Fex_dpto']
['id', 'Orden', 'Clase', 'Dominio', 'P6020', 'P6040', 'P6050', 'P6090', 'P6100', 'P6210', 'P6210s1', 'P6240', 'Oficio', 'P6426', 'P6430', 'P6590', 'P6800', 'P6870', 'P6920', 'P7090', 'P7110', 'P7120', 'P7350', 'P7495', 'P7510s5', 'Pet', 'Oc', 'Des', 'Ina', 'Fex_c', 'Depto', 'Fex_dpto']
['id', 'Clase', 'Dominio', 'P5000', 'P5010', 'P5090', 'P5100', 'P5130', 'P5140', 'Nper', 'Npersug', 'Li', 'Lp', 'Fex_c', 'Depto', 'Fex_dpto']
['id', 'Clase', 'Dominio', 'P5000', 'P5010', 'P5090', 'P5100', 'P5130', 'P5140', 'Nper', 'Npersug', 'Li', 'Lp', 'Fex_c', 'Depto', 'Fex_dpto']


## Fusionar individuos con hogares

In [5]:
# Crear una nueva variable que sea promedio educativo por hogar

# Agrupar por 'id' y sumar los valores de 'P6210s1' para cada hogar
promedio_educacion_train = train_personas.groupby('id')['P6210s1'].sum().reset_index()
promedio_educacion_train.rename(columns={'P6210s1': 'Suma_P6210s1'}, inplace=True)

# Unir con el DataFrame de hogares en base al 'id'
train_hogares = train_hogares.merge(promedio_educacion_train, on='id', how='left')

# Calcular el promedio de educación por hogar
train_hogares['Promedio_Educacion'] = train_hogares['Suma_P6210s1'] / train_hogares['Nper']

# Agrupar por 'id' y sumar los valores de 'P6210s1' para cada hogar
promedio_educacion_test = test_personas.groupby('id')['P6210s1'].sum().reset_index()
promedio_educacion_test.rename(columns={'P6210s1': 'Suma_P6210s1'}, inplace=True)

# Unir con el DataFrame de hogares en base al 'id'
test_hogares = test_hogares.merge(promedio_educacion_test, on='id', how='left')

# Calcular el promedio de educación por hogar
test_hogares['Promedio_Educacion'] = test_hogares['Suma_P6210s1'] / test_hogares['Nper']

## Estadistica descriptiva

In [5]:
# Aca generar estaditica descriptiva (cuando tengamos todos los datos unificados)

## Modelos